In [1]:
import scipy.io as scio
import numpy as np
import math

In [3]:
data = [90, 60, 100, 95, 85, 70, 80, 110, 70]
data = [90, 60, 50, 95, 85, 70, 80, 110, 70]

In [7]:
m = 2
r = 0.2000000001
#data = [0.1,0.2,0.3,0.4,0.5,0.6]#,0.7,0.8,0.9]
N = len(data)
r = 20   #use 100 times of original value, so we could use integer type data

In [8]:
#data = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.15]#,1.2,1.3,1.35]
#data = [0.8,0.9,0.7,0.76,0.78,0.79]
#data =  [-2.2907  ,  0.4626  ,  3.8283  ,  0.9417  ,  0.8600  ,  0.9760  ,  1.0460]
'''software algorithm'''
N = len(data)
x1=[]
x2 = []
import math
for i in range(N-m+1):
    x1.append(list(data[i:i+m]))
for i in range(N-m):
    x2.append(list(data[i:i+m+1]))

count1 = 0
for i in range(N-m+1):
    x1temp = np.array(x1[i])
    for j in range(N-m+1):
        if max(abs(x1temp-np.array(x1[j])))<=r:
            count1 = count1+1
    count1 = count1 - 1
B = count1/((N-m)*(N-m+1))

count2 = 0
for i in range(N-m):
    x2temp = np.array(x2[i])
    for j in range(N-m):
        if max(abs(x2temp-np.array(x2[j])))<=r:
            count2 = count2+1
    count2 = count2 - 1
A = count2/((N-m-1)*(N-m))

math.log(B/A),B,A

(1.6582280766035324, 0.25, 0.047619047619047616)

In [9]:
#simulation for FPGA

D = [[0 for i in range(len(data))] for j in range(len(data))]
for i in range(len(data)):
    for j in range(len(data)):
        if abs(data[i]-data[j])<=r:
            D[i][j] = 1
count1 = 0
for i in range(N-m+1):
    for j in range(N-m+1):
        count1 = count1 + (D[i][j] and D[i+1][j+1])
count1 = count1 - N + m - 1
B = count1/((N-m+1)*(N-m))
count2 = 0
for i in range(N-m):
    for j in range(N-m):
        count2 = count2 + (D[i][j] and D[i+1][j+1] and D[i+2][j+2])
count2 = count2 - N+ m
A = count2/((N-m)*(N-m-1))
math.log(B/A),B,A,count1,count2

(1.6582280766035324, 0.25, 0.047619047619047616, 14, 2)

In [10]:
#https://en.wikipedia.org/wiki/Sample_entropy
import numpy as np

def sampen(L, m, r):
    N = len(L)
    B = 0.0
    A = 0.0
    
    
    # Split time series and save all templates of length m
    xmi = np.array([L[i : i + m] for i in range(N - m)])
    xmj = np.array([L[i : i + m] for i in range(N - m + 1)])

    # Save all matches minus the self-match, compute B
    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) - 1 for xmii in xmi])

    # Similar for computing A
    m += 1
    xm = np.array([L[i : i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r) - 1 for xmi in xm])

    # Return SampEn
    return -np.log(A / B)